In [1]:

def function_1(input_1):
    return input_1 + " First Function "

def function_2(input_2):
    return input_2 + "to Second Function"

In [2]:
from langgraph.graph import Graph

# Define a Langchain graph
workflow = Graph()

workflow.add_node("node_1", function_1)
workflow.add_node("node_2", function_2)
workflow.add_edge('node_1', 'node_2')
workflow.set_entry_point("node_1")
workflow.set_finish_point("node_2")
app = workflow.compile()


In [3]:

app.invoke('I am moving from')


'I am moving from First Function to Second Function'

In [4]:
input = "I am moving from"
for output in app.stream(input):
    for key, value in output.items():
        print(f"output from the node '{key}':")
        print("----")
        print(value)
    print("\n--\n")

output from the node 'node_1':
----
I am moving from First Function 

--

output from the node 'node_2':
----
I am moving from First Function to Second Function

--



# Integrating LLM call in the LangGraph

In [6]:
from dotenv import load_dotenv
load_dotenv()

True

In [9]:
from langchain_groq import ChatGroq
llm=ChatGroq(model_name="qwen-2.5-32b")
llm.invoke("hi")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 30, 'total_tokens': 40, 'completion_time': 0.05, 'prompt_time': 0.003675701, 'queue_time': 0.054126363, 'total_time': 0.053675701}, 'model_name': 'qwen-2.5-32b', 'system_fingerprint': 'fp_35f92f8282', 'finish_reason': 'stop', 'logprobs': None}, id='run-7fdd743b-5f63-4309-85d7-4eef85d9f39d-0', usage_metadata={'input_tokens': 30, 'output_tokens': 10, 'total_tokens': 40})

In [10]:

def function_1(input_1):
    complete_query = "Your task is to provide only the topic based on the user query. \
        Only output the topic among: [Japan , Sports]. Don't include reasoning. Following is the user query: " + input_1
    response = llm.invoke(complete_query)
    return response.content

def function_2(input_2):
    TOPIC_UPPER = input_2.upper()
    response = f"Here is the topic in UPPER case: {TOPIC_UPPER}"
    return response

In [11]:

# Define a Langchain graph
workflow = Graph()

workflow.add_node("Agent", function_1)
workflow.add_node("tool", function_2)

workflow.add_edge('Agent', 'tool')

workflow.set_entry_point("Agent")
workflow.set_finish_point("tool")


app = workflow.compile()


In [12]:
query = "Tell me about Japan's Industrial Growth"
app.invoke(query)

'Here is the topic in UPPER case: JAPAN'

In [13]:
for output in app.stream(query):
    # stream() yields dictionaries with output keyed by node name
    for key, value in output.items():
        print(f"Output from node '{key}':")
        print("---")
        print(value)
    print("\n---\n")

Output from node 'Agent':
---
Japan

---

Output from node 'tool':
---
Here is the topic in UPPER case: JAPAN

---



# RAG Pipeline integration

In [14]:
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

In [18]:
### Reading the txt files from source directory


loader = DirectoryLoader('D:\LangGraph\Langraph-tutorial\LangGraph\data', glob="./*.txt", loader_cls=TextLoader)
docs = loader.load()

### Creating Chunks using RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=10,
    length_function=len
)
new_docs = text_splitter.split_documents(documents=docs)
doc_strings = [doc.page_content for doc in new_docs]

In [20]:
# ###  BGE Embddings

# from langchain.embeddings import HuggingFaceBgeEmbeddings

# model_name = "BAAI/bge-base-en-v1.5"
# model_kwargs = {'device': 'cpu'}
# encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity
# embeddings = HuggingFaceBgeEmbeddings(
#     model_name=model_name,
#     model_kwargs=model_kwargs,
#     encode_kwargs=encode_kwargs,
# )

In [21]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [23]:
db = Chroma.from_documents(new_docs, embeddings)
retriever = db.as_retriever(search_kwargs={"k": 4})

In [25]:

query = "Tell me about japan"
docs = retriever.invoke(query)
print(docs)

[Document(metadata={'source': 'D:\\LangGraph\\Langraph-tutorial\\LangGraph\\data\\japan.txt'}, page_content="Japan's last four year GDP:"), Document(metadata={'source': 'D:\\LangGraph\\Langraph-tutorial\\LangGraph\\data\\japan.txt'}, page_content='Industrial revival hope for Japan'), Document(metadata={'source': 'D:\\LangGraph\\Langraph-tutorial\\LangGraph\\data\\japan.txt'}, page_content="Japanese industry is growing faster than expected, boosting hopes that the country's retreat back"), Document(metadata={'source': 'D:\\LangGraph\\Langraph-tutorial\\LangGraph\\data\\japan.txt'}, page_content="exports, normally the engine for Japan's economy in the face of weak domestic demand, had helped")]


In [26]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain.chains import RetrievalQA

In [27]:

# assign AgentState as an empty dict
AgentState = {}

# messages key will be assigned as an empty array. We will append new messages as we pass along nodes. 
AgentState["messages"] = []

In [28]:

AgentState

{'messages': []}

In [29]:

def function_1(state):
    messages = state['messages']
    question = messages[-1]   ## Fetching the user question
    
    complete_query = "Your task is to provide only the topic based on the user query. \
        Only output the topic among: [Japan , Sports]. Don't include reasoning. Following is the user query: " + question
    response = llm.invoke(complete_query)
    state['messages'].append(response.content) # appending LLM call response to the AgentState
    return state

In [30]:
def function_2(state):
    messages = state['messages']
    question = messages[0] ## Fetching the user question

    template = """Answer the question based only on the following context:
    {context}

    Question: {question}
    """
    prompt = ChatPromptTemplate.from_template(template)

    retrieval_chain = (
        {"context": retriever, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
        )
    result = retrieval_chain.invoke(question)
    return result

In [31]:
# Define a Langchain graph
workflow = Graph()

workflow.add_node("Agent", function_1)
workflow.add_node("tool", function_2)

workflow.add_edge('Agent', 'tool')

workflow.set_entry_point("Agent")
workflow.set_finish_point("tool")

app = workflow.compile()

In [33]:

inputs = {"messages": ["Tell me about japan"]}
app.invoke(inputs)

"Based on the information provided, Japan is currently experiencing positive economic and industrial growth. There is an indication of industrial revival, with the industry growing faster than expected. This growth is contributing to the hope that Japan's economic retreat might be reversing. Additionally, despite weak domestic demand, exports have been a significant contributor to the economy, acting as a key engine for economic activity. However, specific details about Japan's last four years of GDP are mentioned but not detailed in the given context."

In [34]:
inputs = {"messages": ["tell me about gdp of japan"]}
app.invoke(inputs)

"Based on the provided context, here is the information about Japan's GDP for the last four years:\n\n- **2022:** $4,256.41B\n- **2021:** $5,034.62B\n- **2020:** $5,055.59B\n- **2019:** $5,117.99B\n\nThese figures represent the Gross Domestic Product (GDP) of Japan for the specified years."

In [35]:
for output in app.stream(inputs):
    # stream() yields dictionaries with output keyed by node name
    for key, value in output.items():
        print(f"Output from node '{key}':")
        print("---")
        print(value)
    print("\n---\n")

Output from node 'Agent':
---
{'messages': ['tell me about gdp of japan', 'Japan', 'Japan']}

---

Output from node 'tool':
---
Based on the information provided, here is a summary of Japan's GDP over the last four years:

- **2019**: $5,117.99B
- **2020**: $5,055.59B
- **2021**: $5,034.62B
- **2022**: $4,256.41B

The data indicates a general trend of decreasing GDP from 2019 to 2022, with 2022 showing a particularly notable drop.

---



# Simplyfying the State addition and maintainence